In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
import datetime

In [33]:
df=pd.read_csv("credit_card_fraud_dataset.csv")
df.head()

,TransactionID,TransactionDate,Amount,MerchantID,TransactionType,Location,IsFraud
0,1,2024-04-03 14:15:35.462794,4189.27,688,refund,San Antonio,0
1,2,2024-03-19 13:20:35.462824,2659.71,109,refund,Dallas,0
2,3,2024-01-08 10:08:35.462834,784.00,394,purchase,New York,0
3,4,2024-04-13 23:50:35.462850,3514.40,944,purchase,Philadelphia,0
4,5,2024-07-12 18:51:35.462858,369.07,475,purchase,Phoenix,0


In [35]:
print(df.columns)

Index(['TransactionID', 'TransactionDate', 'Amount', 'MerchantID',
       'TransactionType', 'Location', 'IsFraud'],
      dtype='object')


In [37]:
df = df.drop(columns=['TransactionID', 'TransactionDate', 'MerchantID', 'Location'])
df

,Amount,TransactionType,IsFraud
0,4189.27,refund,0
1,2659.71,refund,0
2,784.00,purchase,0
3,3514.40,purchase,0
4,369.07,purchase,0
...,...,...,...
99995,1057.29,refund,0
99996,297.25,refund,0
99997,3448.56,purchase,0
99998,3750.79,purchase,0


In [39]:
df = pd.get_dummies(df, columns=['TransactionType'], drop_first=True)
df

,Amount,IsFraud,TransactionType_refund
0,4189.27,0,True
1,2659.71,0,True
2,784.00,0,False
3,3514.40,0,False
4,369.07,0,False
...,...,...,...
99995,1057.29,0,True
99996,297.25,0,True
99997,3448.56,0,False
99998,3750.79,0,False


In [41]:
X = df.drop('IsFraud', axis=1)
y = df['IsFraud']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [43]:
from collections import Counter

print("Before SMOTE:", Counter(y_train))

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("After SMOTE:", Counter(y_train_res))

Before SMOTE: Counter({0: 79200, 1: 800})
After SMOTE: Counter({0: 79200, 1: 79200})


In [45]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [47]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train_res, y_train_res,
    epochs=10,
    validation_data=(X_test, y_test),
    callbacks=[tensorboard_callback]
)

C:\Users\lenovo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
4950/4950 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - accuracy: 0.5148 - loss: 0.6931 - val_accuracy: 0.4742 - val_loss: 0.6847
Epoch 2/10
4950/4950 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.5222 - loss: 0.6923 - val_accuracy: 0.3717 - val_loss: 0.7088
Epoch 3/10
4950/4950 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.5234 - loss: 0.6921 - val_accuracy: 0.4731 - val_loss: 0.6964
Epoch 4/10
4950/4950 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.5261 - loss: 0.6918 - val_accuracy: 0.4324 - val_loss: 0.6819
Epoch 5/10
4950/4950 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.5269 - loss: 0.6915 - val_accuracy: 0.4852 - val_loss: 0.6776
Epoch 6/10
4950/4950 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.5293 - loss: 0.6906 - val_accuracy: 0.4565 - val_loss: 0.6908
Epoch 7/10
4950/4950 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.5303 - loss: 0.6907 - val_accuracy: 0.4058 - val_loss: 0.7003
Epoch 8/10
4950/4950 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.5300 - loss: 0.6906 

In [49]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.59      0.74     19800
           1       0.01      0.35      0.02       200

    accuracy                           0.59     20000
   macro avg       0.50      0.47      0.38     20000
weighted avg       0.98      0.59      0.74     20000

Confusion Matrix:
 [[11763  8037]
 [  129    71]]


In [51]:
%load_ext tensorboard
%tensorboard --logdir logs/fit
